# Fussing with the BiasFrame class [v1.4]

In [1]:
# imports
import glob
import os

from importlib import reload
from pypit import biasframe

from pypit.par import pypitpar

In [2]:
pypdev_path = os.getenv('PYPIT_DEV')

In [3]:
## Additional settings
#kast_settings['reduce'] = {}
#kast_settings['reduce']['masters'] = {}
#kast_settings['reduce']['masters']['reuse'] = False
#kast_settings['reduce']['masters']['force'] = False
#kast_settings['reduce']['masters']['loaded'] = []
##
#kast_settings['run'] = {}
#kast_settings['run']['spectrograph'] = 'shane_kast_blue'
#kast_settings['run']['directory'] = {}
#kast_settings['run']['directory']['master'] = 'MF'
#setup = 'A_01_aa'

# Would set masters='reuse' for settings['reduce']['masters']['reuse'] = True or 
# masters='force' for settings['reduce']['masters']['force'] = True.
# In the actual pypit run these would get set from a config file.
# Maybe should combine 'run' and 'reduce'?
runpar = pypitpar.RunPar(caldir='MF')
rdxpar = pypitpar.ReducePar(spectrograph='shane_kast_blue', masters=None, setup='A_01_aa')

## Generate Bias Image

In [4]:
kast_blue_bias = glob.glob(os.path.join(os.getenv('PYPIT_DEV'), 'RAW_DATA',
                                        'Shane_Kast_blue', '600_4310_d55', 'b1?.fits*'))
kast_blue_bias

['/Users/westfall/Work/packages/PYPIT-development-suite/RAW_DATA/Shane_Kast_blue/600_4310_d55/b10.fits.gz',
 '/Users/westfall/Work/packages/PYPIT-development-suite/RAW_DATA/Shane_Kast_blue/600_4310_d55/b11.fits.gz',
 '/Users/westfall/Work/packages/PYPIT-development-suite/RAW_DATA/Shane_Kast_blue/600_4310_d55/b12.fits.gz',
 '/Users/westfall/Work/packages/PYPIT-development-suite/RAW_DATA/Shane_Kast_blue/600_4310_d55/b13.fits.gz',
 '/Users/westfall/Work/packages/PYPIT-development-suite/RAW_DATA/Shane_Kast_blue/600_4310_d55/b14.fits.gz',
 '/Users/westfall/Work/packages/PYPIT-development-suite/RAW_DATA/Shane_Kast_blue/600_4310_d55/b15.fits.gz',
 '/Users/westfall/Work/packages/PYPIT-development-suite/RAW_DATA/Shane_Kast_blue/600_4310_d55/b16.fits.gz',
 '/Users/westfall/Work/packages/PYPIT-development-suite/RAW_DATA/Shane_Kast_blue/600_4310_d55/b17.fits.gz',
 '/Users/westfall/Work/packages/PYPIT-development-suite/RAW_DATA/Shane_Kast_blue/600_4310_d55/b18.fits.gz',
 '/Users/westfall/Work/packa

### Instantiate

In [5]:
# Change one default just to make sure it propagates 
biaspar = pypitpar.FrameGroupPar(frametype='bias',
                                 combine=pypitpar.CombineFramesPar(cosmics=25.))
print(biaspar['combine']['cosmics'])
print(biaspar.default['combine']['cosmics'])
print(biaspar['useframe'])

25.0
20.0
None


In [6]:
#reload(biasframe)
root_path = os.path.join(os.getcwd(), runpar['caldir'])
bias_frame = biasframe.BiasFrame(rdxpar['spectrograph'], file_list=kast_blue_bias,
                                 par=biaspar, setup=rdxpar['setup'], root_path=root_path,
                                 mode=rdxpar['masters'])

In [7]:
bias_frame

<BiasFrame: nimg=10>

In [8]:
bias_frame.directory_path

'/Users/westfall/Work/packages/PYPIT/doc/nb/MF_shane_kast_blue'

In [9]:
bias_frame.spectrograph.detector[0]['datasec']

['[1:1024,:]', '[1025:2048,:]']

In [10]:
print(bias_frame.combine_par['cosmics'])
print(bias_frame.par['combine']['cosmics'])
print(bias_frame.combine_par['cosmics'] is bias_frame.par['combine']['cosmics'])

25.0
25.0
True


### Process

In [11]:
bias_img = bias_frame.process()

[WARNING] :: Your images have not been bias subtracted!
[INFO]    :: Combining 10 bias frames
[INFO]    :: Finding saturated and non-linear pixels
[INFO]    :: Rejecting cosmic rays
[INFO]    :: Not rejecting any low/high pixels
[INFO]    :: Rejecting deviant pixels
[INFO]    :: Combining frames with a mean operation
[INFO]    :: Replacing completely masked pixels with the maxnonsat value of the input frames
[INFO]    :: 10 bias frames combined successfully!


In [12]:
bias_frame.steps

['load_images', 'combine']

In [13]:
#bias_frame.show('stack')
# (KBW) this was causing me problems...

### Write
(KBW) Are writing/loading used in this way anymore?

In [14]:
#bias_frame.write_stack_to_fits('tmp.fits')

### Load from disk

In [15]:
#bias_frame2 = biasframe.BiasFrame.from_fits('tmp.fits')
#bias_frame2.stack.shape

## Run (from scratch) as called from PYPIT

### Creates bias and saves as a MasterFrame to MF_shane_kast_blue/  which I needed to generate

In [16]:
# Instantiate
bias_frame2 = biasframe.BiasFrame(rdxpar['spectrograph'], file_list=kast_blue_bias,
                                  par=biaspar, setup=rdxpar['setup'], root_path=root_path,
                                  mode=rdxpar['masters'])

In [17]:
try:
    os.mkdir('MF_shane_kast_blue')
except:
    pass
bias = bias_frame2.build_image()

[WARNING] :: Your images have not been bias subtracted!
[INFO]    :: Combining 10 bias frames
[INFO]    :: Finding saturated and non-linear pixels
[INFO]    :: Rejecting cosmic rays
[INFO]    :: Not rejecting any low/high pixels
[INFO]    :: Rejecting deviant pixels
[INFO]    :: Combining frames with a mean operation
[INFO]    :: Replacing completely masked pixels with the maxnonsat value of the input frames
[INFO]    :: 10 bias frames combined successfully!


In [18]:
bias_frame2.steps

['load_images', 'combine']

In [19]:
# Save
bias_frame2.save_master(bias)

[INFO]    :: Saving master bias frame as:
             /Users/westfall/Work/packages/PYPIT/doc/nb/MF_shane_kast_blue/MasterBias_A_01_aa.fits
[INFO]    :: Writing header information
[INFO]    :: Master bias frame saved successfully:
             /Users/westfall/Work/packages/PYPIT/doc/nb/MF_shane_kast_blue/MasterBias_A_01_aa.fits


## Load Master (several ways)

### master()

In [20]:
biaspar['useframe'] = 'bias'
bias_frame3 = biasframe.BiasFrame(rdxpar['spectrograph'], file_list=kast_blue_bias,
                                  par=biaspar, setup=rdxpar['setup'], root_path=root_path,
                                  mode='reuse')
bias3 = bias_frame3.master()

[INFO]    :: Loading a pre-existing master calibration frame


In [21]:
bias3.shape

(2112, 350)

### Direct load of all master frame stuff

In [22]:
bias4, _, _ = bias_frame3.load_master_frame()
bias4.shape

[INFO]    :: Loading a pre-existing master calibration frame


(2112, 350)

## Clean up

In [23]:
os.remove('MF_shane_kast_blue/MasterBias_A_01_aa.fits')
os.rmdir('MF_shane_kast_blue')